In [ ]:
import email
import re
import sys
from email.header import decode_header

import chardet
import jieba
from bs4 import BeautifulSoup
from keras.models import load_model
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from zhconv import convert

In [ ]:
NOT_ALLOW_CHAR = re.compile(u"[^\u4e00-\u9fffa-zA-Z,.\s]")
SYMBOL_RE = re.compile(u"[.,\[\]@\s:()\r\n\-]+")
SPACE_RE = re.compile("^\s+$")
ENCODING_RE = re.compile(r"content-type(.*?);(\s+)?charset(\s+)?=(\s+)?['\"]?(.*?)['\"]?\s",
                         re.IGNORECASE | re.MULTILINE)

eml_path = "Y:\\085.txt" # if len(sys.argv) != 3 else sys.argv[2]
write_path = "zxcv"
model_path = "m.h5"

In [ ]:
def split_words(txt: str):
    txt = re.sub(SYMBOL_RE, " ", txt)
    txt = re.sub(NOT_ALLOW_CHAR, "", txt)
    txt = txt.strip()
    iterator = jieba.cut(txt)
    return list(filter(lambda s: SYMBOL_RE.match(s) is None, iterator))


def get_mail_content(msg):
    if isinstance(msg, list):
        return ' '.join(map(get_mail_content, msg))
    else:
        content_charset = msg.get_content_charset()
        content_type = msg.get_content_type()
        content_text = ''
        try:
            if msg.get('content-transfer-encoding') is not None:
                content_text = msg.get_payload(decode=True).decode(content_charset)
            else:
                content_text = msg.get_payload(decode=False)
        except:
            content_text = msg.get_payload(decode=False)

        if content_type == 'text/plain':
            return content_text
        elif content_type == 'text/html':
            html_instance = BeautifulSoup(content_text, 'html5lib')
            return str(html_instance.getText())
        else:
            payload = msg.get_payload(decode=False)
            return payload


def read_email(file_path):
    fp = open(file_path, 'rb')
    try:
        eml_content_bytes = fp.read()
        fp.close()
        guessed_encoding = chardet.detect(eml_content_bytes)
        used_encoding = guessed_encoding['encoding']
        if guessed_encoding['confidence'] < 0.87:
            m = ENCODING_RE.search(str(eml_content_bytes, 'ascii', errors='ignore'))
            if m is not None:
                m_list = m.groups()
                if len(m_list) >= 5:
                    used_encoding = m_list[4]

            if used_encoding is None:
                used_encoding = 'utf-8'

        eml_content = str(eml_content_bytes, encoding=used_encoding, errors='ignore')
        return email.message_from_string(eml_content)
    except Exception as e:
        print(file_path, e)
        return None



In [ ]:
text_list = []
msg = read_email(eml_path)

In [ ]:
subject, charset = decode_header(msg['SUBJECT'])[0]

if charset:
    subject = subject.decode(charset, errors='ignore')

subject = split_words(subject)
print(subject)
text_list.extend(subject)

In [ ]:
content = ""
if msg.is_multipart():
    content = get_mail_content(msg.get_payload())
else:
    content = get_mail_content(msg)

print(content)
results = split_words(content)
print(results)
text_list.extend(results)

In [ ]:
cn_text_list = list(map(lambda txt: convert(txt, 'zh-cn'), text_list))
print(cn_text_list)
# print(HanziConv.toTraditional(str))
seg_list = cn_text_list

In [ ]:
with open(write_path, "w+", encoding='utf8') as f:
    f.write("/ ".join(seg_list))

In [ ]:
model = load_model(model_path)


In [ ]:
s = []
k = ''
with open(write_path, 'r', encoding='utf8') as f:
    k = f.read().replace("\n", "/").replace(" ", "").split("/")
while True:
    k_index = k.index('') if '' in k else -1
    if k_index == -1:
        break
    k.pop(k_index)
print(k)
s += [k]

templ = len(s)
tokenizer = Tokenizer()
tokenizer.fit_on_texts(list(s))
sequences = tokenizer.texts_to_sequences(s)
data = pad_sequences(sequences, maxlen=200)
print(sequences)
print(data)

y = model.predict_classes(data)

print('spam' if (y[0] == 0) else 'ham')
# category={'spam': 0, 'ham': 1}
